# Setting of Snowpipe

## KAPAに対するSNowpipeの設定

In [ ]:
use role sysadmin;

CREATE OR REPLACE PIPE kapa_raw_pipe
  AUTO_INGEST = TRUE
  AS
  COPY INTO KAPA_RAW (
    raw_data,
    source_file_name,
    source_file_row_number,
    load_timestamp,
    etl_batch_id
)
FROM (
    SELECT 
        $1 as raw_data,                            -- 元のJSONデータ
        METADATA$FILENAME as source_file_name,     -- ソースファイル名
        METADATA$FILE_ROW_NUMBER as source_file_row_number, -- 行番号
        CURRENT_TIMESTAMP() as load_timestamp,     -- ロード時刻
        'KAPA_' || TO_VARCHAR(CURRENT_TIMESTAMP(), 'YYYYMMDD_HH24MISS') as etl_batch_id
    FROM @CAPSTONE.PUBLIC.CAPSTONE_S3_STAGE/kapa-0001/
)
FILE_FORMAT = (
    TYPE = 'JSON',
    COMPRESSION = 'GZIP',
    STRIP_OUTER_ARRAY = TRUE
)
ON_ERROR = CONTINUE; -- エラーがあっても処理を継続


### Snowflakeで通知チャネルARNを取得

In [ ]:
DESCRIBE PIPE kapa_raw_pipe;

In [ ]:
-- 例: パイプの所有者が MY_CUSTOM_ROLE だった場合
GRANT USAGE ON STAGE CAPSTONE.PUBLIC.CAPSTONE_S3_STAGE TO ROLE sysadmin;

In [ ]:
-- INSERT OVERWRITE INTO KAPA_RAW 
-- SELECT * FROM KAPA_RAW AT (OFFSET => -60*15);

### AWSでS3イベント通知を設定:
AWSの管理コンソールで、対象のS3バケットの「プロパティ」タブにある「イベント通知」を設定します。

- イベントタイプ: s3:ObjectCreated:All (すべてのオブジェクト作成イベント) を選択します。

- 送信先: 「SQSキュー」を選択し、ステップ1でコピーしたARN（arn:aws:sqs:ap-northeast-1:392051336358:sf-snowpipe-AIDAVWSAUUSTELF5LPBXX-MuSWT-kb32x-404jKikLdg）を貼り付けます。

## KFBIに対するSnowpipeの設定


In [ ]:
use role sysadmin;

CREATE OR REPLACE PIPE kfbi_raw_pipe
  AUTO_INGEST = TRUE
  AS
    -- KBFI（旧ファームウェア）データのロード
    COPY INTO KBFI_RAW (
        raw_data,
        source_file_name,
        source_file_row_number,
        load_timestamp,
        etl_batch_id
    )
    FROM (
        SELECT 
            $1 as raw_data,                               -- 元のJSONデータ
            METADATA$FILENAME as source_file_name,        -- ソースファイル名
            METADATA$FILE_ROW_NUMBER as source_file_row_number, -- 行番号
            CURRENT_TIMESTAMP() as load_timestamp,        -- ロード時刻
            'KBFI_' || TO_VARCHAR(CURRENT_TIMESTAMP(), 'YYYYMMDD_HH24MISS') as etl_batch_id
        FROM @CAPSTONE.PUBLIC.CAPSTONE_S3_STAGE/kbfi-0001/            -- 実際のステージ名に変更
    )
    FILE_FORMAT = (
        TYPE = 'JSON'
        STRIP_OUTER_ARRAY = FALSE
        COMPRESSION = AUTO
    )
    ON_ERROR = CONTINUE  

In [ ]:
DESCRIBE PIPE kfbi_raw_pipe;

In [ ]:
-- 例: パイプの所有者が sysadmin だった場合
GRANT USAGE ON STAGE CAPSTONE.PUBLIC.CAPSTONE_S3_STAGE TO ROLE sysadmin;